In [1]:
import pandas as pd
import numpy as np
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [2]:
h2o.init()

train = h2o.import_file('./train.csv')
test = h2o.import_file('./test.csv')
sample_submission = h2o.import_file('./gender_submission.csv')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.11+9-LTS-194, mixed mode)
  Starting server from C:\Users\Gerd\venvs\h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Gerd\AppData\Local\Temp\tmpg0kodm88
  JVM stdout: C:\Users\Gerd\AppData\Local\Temp\tmpg0kodm88\h2o_Gerd_started_from_python.out
  JVM stderr: C:\Users\Gerd\AppData\Local\Temp\tmpg0kodm88\h2o_Gerd_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,2 months and 17 days
H2O_cluster_name:,H2O_from_python_Gerd_5zh9qp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.975 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
#drop the cabin column as it contains alot of missing values
#train = train.drop(['Cabin'], axis = 1)
train = train.drop(['PassengerId'], axis = 1)


predictors = list(train.columns)
predictors.remove('Survived')
predictors.remove('Name')
predictors.remove('Ticket')
predictors.remove('Cabin')

target = 'Survived'

train["Survived"] = train["Survived"].asfactor()

In [4]:
train_split, test_split, valid_split = train.split_frame([0.8, 0.1])

In [5]:
predictors

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [6]:
glm_model = H2OGeneralizedLinearEstimator(family= "binomial",
                                          model_id = "titanic-glm",
                                         alpha=1)

glm_model.train(predictors, target, training_frame=train_split, validation_frame=valid_split)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [8]:
glm_model.model_performance(test_split)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.11175540620239797
RMSE: 0.3342983790005539
LogLoss: 0.3769642389212399
Null degrees of freedom: 96
Residual degrees of freedom: 86
Null deviance: 129.89188302784444
Residual deviance: 73.13106235072053
AIC: 95.13106235072053
AUC: 0.8847011596788581
AUCPR: 0.8907296714745477
Gini: 0.7694023193577162

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4846911418522008: 


,,0,1,Error,Rate
0,0,54.0,5.0,0.0847,(5.0/59.0)
1,1,8.0,30.0,0.2105,(8.0/38.0)
2,Total,62.0,35.0,0.134,(13.0/97.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.484691,0.821918,34.0
1,max f2,0.196999,0.833333,50.0
2,max f0point5,0.591020,0.876623,28.0
3,max accuracy,0.591020,0.865979,28.0
4,max precision,0.953803,1.000000,0.0
5,max recall,0.058723,1.000000,82.0
6,max specificity,0.953803,1.000000,0.0
7,max absolute_mcc,0.591020,0.721463,28.0
8,max min_per_class_accuracy,0.330099,0.842105,39.0
9,max mean_per_class_accuracy,0.484691,0.852364,34.0



Gains/Lift Table: Avg response rate: 39,18 %, avg score: 38,41 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010309,0.950167,2.552632,2.552632,1.000000,0.953803,1.000000,0.953803,0.026316,0.026316,155.263158,155.263158,0.026316
1,2,0.020619,0.937869,2.552632,2.552632,1.000000,0.950016,1.000000,0.951910,0.026316,0.052632,155.263158,155.263158,0.052632
2,3,0.030928,0.936165,2.552632,2.552632,1.000000,0.936813,1.000000,0.946877,0.026316,0.078947,155.263158,155.263158,0.078947
3,4,0.041237,0.933628,2.552632,2.552632,1.000000,0.936077,1.000000,0.944177,0.026316,0.105263,155.263158,155.263158,0.105263
4,5,0.051546,0.921494,2.552632,2.552632,1.000000,0.933162,1.000000,0.941974,0.026316,0.131579,155.263158,155.263158,0.131579
5,6,0.103093,0.895059,2.552632,2.552632,1.000000,0.907274,1.000000,0.924624,0.131579,0.263158,155.263158,155.263158,0.263158
6,7,0.154639,0.870209,2.552632,2.552632,1.000000,0.886049,1.000000,0.911766,0.131579,0.394737,155.263158,155.263158,0.394737
7,8,0.206186,0.777864,2.042105,2.425000,0.800000,0.827301,0.950000,0.890650,0.105263,0.500000,104.210526,142.500000,0.483051
8,9,0.298969,0.565693,2.269006,2.376588,0.888889,0.704314,0.931034,0.832821,0.210526,0.710526,126.900585,137.658802,0.676628
9,10,0.402062,0.347465,1.021053,2.029015,0.400000,0.490324,0.794872,0.745002,0.105263,0.815789,2.105263,102.901484,0.680196


In [9]:
predictions = glm_model.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [10]:
p = predictions.as_data_frame()
t = test.as_data_frame()

In [11]:
result = pd.concat([t, p], axis=1)
result  = result[['PassengerId','predict']]
#result['Survived'] = np.where(result['predict'] >= 0.5, 1, 0)
result['Survived'] = result['predict']
result = result.drop(['predict'], axis = 1)

In [12]:
result.to_csv('./my_submission.csv', index=False)

In [13]:
h2o.cluster().shutdown()

H2O session _sid_b918 closed.
